In [7]:
import duckdb
import pandas as pd  
import os
from datetime import datetime

In [9]:
#apagar dados_duckdb.db
if os.path.exists('dados_duckdb.db'):
    os.remove('dados_duckdb.db')

PermissionError: [WinError 32] O arquivo já está sendo usado por outro processo: 'dados_duckdb.db'